Effets des variables macroéconomiques sur le CAC 40

In [358]:
import pandas as pd
import numpy as np
from datetime import datetime
!pip install openpyxl

Chargement des fichiers Excel qui forment la base de donnée

In [359]:
chomage = pd.read_excel('Chômage_depuis1975.xlsx')
confiance = pd.read_excel('Confiance_ménages.xlsx')
ev_pib = pd.read_excel('Evolution_PIB_trimestre.xlsx')
ipc = pd.read_excel('ipc_menages.xlsx')

/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Nettoyage des données

In [360]:
chomage.head()

,Tableau 1. Taux de chômage trimestriel au sens du BIT en France (hors Mayotte) par sexe et âge,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,"Données CVS en moyenne trimestrielle, en %",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,En jaune : Les séries ont été constituées à pa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,HOMMES (%),NaN,NaN,NaN,FEMMES (%),NaN,NaN,NaN,TOTAL (%),NaN,NaN,NaN,Nombre de chômeurs
4,trimestre.année,Âge,NaN,NaN,Total,Âge,NaN,NaN,Total,Âge,NaN,NaN,Total,NaN


In [361]:
# Renommer les colonnes à utiliser
chomage.rename(columns={'Unnamed: 12': 'taux_chomage'}, inplace=True)
chomage.rename(columns={'Tableau 1. Taux de chômage trimestriel au sens du BIT en France (hors Mayotte) par sexe et âge': 'trimestre'}, inplace=True) 

In [362]:
print(chomage.columns)

Index(['trimestre', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'taux_chomage', 'Unnamed: 13'],
      dtype='object')


In [363]:
# Supprimer les colonnes inutiles
chomage.drop(columns=[ 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 
    'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
    'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 13'], inplace=True)

# Afficher la table après suppression
chomage

,trimestre,taux_chomage
0,"Données CVS en moyenne trimestrielle, en %",NaN
1,NaN,NaN
2,En jaune : Les séries ont été constituées à pa...,NaN
3,NaN,NaN
4,trimestre.année,Total
...,...,...
203,T2.2024,7.3
204,T3.2024,7.4
205,NaN,NaN
206,"Champ : France hors Mayotte, personnes vivant ...",NaN


In [364]:
# Trouver l'index de la valeur 'T1.2000' dans la colonne 'trimestre'
index_t1_2000 = chomage[chomage['trimestre'] == 'T1.2000'].index

# Afficher l'index
print(index_t1_2000)


Index([106], dtype='int64')


In [365]:
# On supprime les premières lignes inutiles pour que les données débutent au premier trimestre  
chomage = chomage.iloc[105:].reset_index(drop=True)

# Afficher les premieres lignes des données nettoyées
chomage.head()


,trimestre,taux_chomage
0,T4.1999,9.5
1,T1.2000,9.1
2,T2.2000,8.7
3,T3.2000,8.4
4,T4.2000,8


In [366]:
#Déterminuer où sont situés les NaN s'il y en a
print(chomage[chomage.isna().any(axis=1)])

                                             trimestre taux_chomage
100                                                NaN          NaN
101  Champ : France hors Mayotte, personnes vivant ...          NaN
102                     Source : Insee, enquête Emploi          NaN


In [367]:
# Supprimer les lignes contenant des NaN car elles ne comportent pas de données
chomage = chomage.dropna()

# Vérifier le résultat
print(chomage.isna().sum())  

trimestre       0
taux_chomage    0
dtype: int64


Nettoyage des données sur la confiance 

In [368]:
confiance.head()

,Opinion des ménages-Monthly confidence consumer survey,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Données corrigées des variations saisonnières-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DATE,Indicateur synthétique,Niveau de vie en France - évolution passée,Niveau de vie en France - perspectives d'évolu...,Chômage - perspectives d'évolution,Prix - évolution passée,Prix - perspectives d'évolution,Opportunité de faire des achats importants,Opportunité d'épargner,Capacité d'épargne actuelle,Situation financière personnelle - évolution p...,Situation financière personnelle - perspective...,Capacité d'épargne future
3,NaN,Synthetic index,"General economic situation, past 12 months","General economic situation, next 12 months","Unemployment, next 12 months","Consumer prices, past 12 months","Consumer prices, next 12 months","Major purchases intentions, next 12 months","Savings intentions, next 12 months",Current saving capacity,"Financial situation, past 12 months","Financial situation, next 12 months",Expected saving capacity
4,moyenne/average,100,-47.749835,-27.733798,32.585969,-13.383858,-31.095762,-15.188091,18.029779,9.656539,-20.980754,-6.560103,-7.110802


In [369]:
# Renommer les colonnes à utiliser
confiance.rename(columns={'Unnamed: 1': 'indic_confiance'}, inplace=True)
confiance.rename(columns={'Opinion des ménages-Monthly confidence consumer survey': 'date'}, inplace=True) 

In [370]:
print(confiance.columns)

Index(['date', 'indic_confiance', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [371]:
# Supprimer les colonnes inutiles pour notre analyse
confiance.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'], inplace=True)

In [372]:
# S'assurer que la colonne 'date' est au format datetime
confiance['date'] = pd.to_datetime(confiance['date'], errors='coerce')

# Filtrer les données à partir de janvier 2000
confiance =confiance[confiance['date'] >= '2000-01-01']

# Afficher les premières lignes des données filtrées
confiance.head()

/tmp/ipykernel_20906/3570054967.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  confiance['date'] = pd.to_datetime(confiance['date'], errors='coerce')


,date,indic_confiance
204,2000-01-01,119.186742
205,2000-02-01,119.010085
206,2000-03-01,119.930046
207,2000-04-01,122.586881
208,2000-05-01,122.625812


In [373]:
# Convertir les données journalières en trimestrielles (moyenne trimestrielle)
confiance["date"] = pd.to_datetime(confiance["date"])
confiance.set_index("date", inplace=True)
confiance = confiance.resample("Q").mean()

#Vérification
confiance.head()

/tmp/ipykernel_20906/1694031925.py:4: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  confiance = confiance.resample("Q").mean()


,indic_confiance
date,
2000-03-31,119.375624
2000-06-30,122.715558
2000-09-30,121.334655
2000-12-31,121.035423
2001-03-31,124.096635


In [374]:
#Déterminuer où sont situés les NaN s'il y en a
print(confiance[confiance.isna().any(axis=1)])

Empty DataFrame
Columns: [indic_confiance]
Index: []


Nettoyage de la table Evolution du PIB

On cherche à isoler des indicateurs pour l'évolution du PIB, la consommation des ménages et des APU, la formation brute de capital fixe, les importations et exportations. On fait le choix de prendre plus d'indicateurs, quitte à les éliminer par la suite.

In [375]:
ev_pib.head()

,Évolution du produit intérieur brut et de ses composantes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,par rapport au trimestre précédent en volume en %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Trimestre,Produit intérieur brut (PIB),Importations,Dépense de consommation des ménages,Dépense de consommation des APU1,Formation brute de capital fixe,dont :,NaN,NaN,Exportations,Contributions :,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Entreprises non financières,ménages,APU1,NaN,Demande intérieure finale hors stocks,Variations de stocks,Commerce extérieur
4,2024-T3,0.362294,-0.684306,0.515766,0.513113,-0.84265,-1.365417,-0.887841,0.472325,-0.470732,0.212544,0.069318,0.080432


In [376]:
# Renommer les colonnes à utiliser
ev_pib.rename(columns={'Évolution du produit intérieur brut et de ses composantes': 'trimestre'}, inplace=True)
ev_pib.rename(columns={'Unnamed: 1': 'ev_pib'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 2': 'ev_importations'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 3': 'ev_conso_menages'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 4': 'ev_conso_apu'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 5': 'ev_fbcf'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 9': 'ev_exportations'}, inplace=True) 

In [377]:
print(ev_pib.columns)

Index(['trimestre', 'ev_pib', 'ev_importations', 'ev_conso_menages',
       'ev_conso_apu', 'ev_fbcf', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'ev_exportations', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [378]:
# Supprimer les colonnes
ev_pib.drop(columns=['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
    'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'], inplace=True)

#Inversion des colonnes pour avoir des données chronologiques
ev_pib = ev_pib.iloc[::-1].reset_index(drop=True)

# Vérification
ev_pib.head()

,trimestre,ev_pib,ev_importations,ev_conso_menages,ev_conso_apu,ev_fbcf,ev_exportations
0,"Source : Insee, comptes nationaux trimestriels...",NaN,NaN,NaN,NaN,NaN,NaN
1,Champ : France.,NaN,NaN,NaN,NaN,NaN,NaN
2,"Lecture : au 3e trimestre 2024, le produit int...",NaN,NaN,NaN,NaN,NaN,NaN
3,Note : données révisées ; les volumes sont mes...,NaN,NaN,NaN,NaN,NaN,NaN
4,1. Administrations publiques.,NaN,NaN,NaN,NaN,NaN,NaN


On remarque que les données sont ordonnées des plus récentes aux plus anciennes, on souhaite inverser cela pour faciliter le travail avec les autres tables. 

In [379]:
# Trouver l'index de la valeur '2000-T1' dans la colonne 'trimestre'
index_2000_t1 = ev_pib[ev_pib['trimestre'] == '2000-T1'].index

# Afficher l'index
print(index_2000_t1)


Index([209], dtype='int64')


In [380]:
# On supprime les premières lignes inutiles pour que les données débutent au premier trimestre de l'année 2000
ev_pib = ev_pib.iloc[209:].reset_index(drop=True)

On souhaite créer une nouvelle variable qui regroupe l'évolution de la consommation des ménages et des APU.

In [381]:
ev_pib['ev_conso'] = ev_pib['ev_conso_menages'] + ev_pib['ev_conso_apu']

# Afficher les premieres lignes des données 
ev_pib.head()

,trimestre,ev_pib,ev_importations,ev_conso_menages,ev_conso_apu,ev_fbcf,ev_exportations,ev_conso
0,2000-T1,1.011252,4.716928,1.256031,0.606497,2.244349,2.839274,1.862528
1,2000-T2,1.06425,4.20051,0.772804,0.464126,0.921508,4.963057,1.23693
2,2000-T3,0.780148,3.529602,0.444674,0.34475,2.53796,1.41363,0.789424
3,2000-T4,0.563163,2.2326,0.132284,0.155773,0.073162,2.984615,0.288057
4,2001-T1,0.57489,-1.295035,1.139891,0.125288,0.672418,0.408129,1.265179


Vérification des NaN

In [382]:
print(ev_pib[ev_pib.isna().any(axis=1)])

                                             trimestre ev_pib ev_importations  \
99                                                 NaN    NaN             NaN   
101  par rapport au trimestre précédent en volume en %    NaN             NaN   
102                                                NaN    NaN             NaN   

    ev_conso_menages ev_conso_apu ev_fbcf ev_exportations ev_conso  
99               NaN          NaN     NaN             NaN      NaN  
101              NaN          NaN     NaN             NaN      NaN  
102              NaN          NaN     NaN             NaN      NaN  


In [383]:
#Suppression des lignes avec des NaN car elles ne nous intéressent pas
ev_pib = ev_pib.dropna()

#Vérification
print(ev_pib.isna().sum())

trimestre           0
ev_pib              0
ev_importations     0
ev_conso_menages    0
ev_conso_apu        0
ev_fbcf             0
ev_exportations     0
ev_conso            0
dtype: int64


Nettoyage des données sur l'indice des prix à la consommation

In [384]:
ipc.head()

,Libellé,Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble,Unnamed: 2
0,idBank,001759970,NaN
1,Dernière mise à jour,29/11/2024 08:45,NaN
2,Période,NaN,Date de parution au JO
3,2024-11,119.81,NaN
4,2024-10,119.89,17/11/2024


In [385]:
# Renommer les colonnes à utiliser
ipc.rename(columns={'Libellé': 'mois_année'}, inplace=True)
ipc.rename(columns={'Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble': 'ipc_menage'}, inplace=True) 

In [386]:
print(ipc.columns)

Index(['mois_année', 'ipc_menage', 'Unnamed: 2'], dtype='object')


In [387]:
# Supprimer les colonnes
ipc.drop(columns='Unnamed: 2', inplace=True)

# On supprime les premières lignes inutiles 
ipc = ipc.iloc[3:].reset_index(drop=True)

#Inversion des données pour les classer en ordre chronologique
ipc = ipc.iloc[::-1].reset_index(drop=True)

# Trouver l'index de la valeur '2000-01' dans la colonne 'mois_année'
index_2000_01 = ipc[ipc['mois_année'] == '2000-01'].index

# Afficher l'index
print(index_2000_01)

Index([120], dtype='int64')


In [388]:
# On supprime les premières lignes pour que les données débutent en janvier 2000
ipc = ipc.iloc[120:].reset_index(drop=True)

ipc.head()

,mois_année,ipc_menage
0,2000-01,79.17
1,2000-02,79.27
2,2000-03,79.64
3,2000-04,79.64
4,2000-05,79.8


In [389]:
sum(ipc.isna)

TypeError: 'method' object is not iterable

Téléchargement des données sur le CAC 40

In [ ]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import yfinance as yf
import pandas as pd

# Téléchargement les données trimestrielles du CAC 40 avec l'API yfinance
CAC40_y = "^FCHI"  # Symbole Yahoo Finance pour le CAC 40
dfcac40 = yf.download(CAC40_y, start="2000-01-01", interval="3mo")

# Sauvegarder en CSV 
dfcac40.to_csv("cac40_quarterly.csv")


[*********************100%***********************]  1 of 1 completed


nettoyage des données du CAC40

In [ ]:
cac40 = pd.read_csv("cac40_quarterly.csv")

In [ ]:
cac40.head()

,Price,Adj Close,Close,High,Low,Open,Volume
0,Ticker,^FCHI,^FCHI,^FCHI,^FCHI,^FCHI,^FCHI
1,Date,NaN,NaN,NaN,NaN,NaN,NaN
2,1999-12-01,6190.9599609375,6190.9599609375,6396.97021484375,5388.85009765625,6024.3798828125,0
3,2000-03-01,6426.259765625,6426.259765625,6591.43994140625,5752.43994140625,6272.1298828125,0
4,2000-06-01,6625.419921875,6625.419921875,6780.66015625,6354.93017578125,6428.4599609375,0


In [ ]:
# Supprimer la colonne Volume
cac40.drop(columns='Volume', inplace=True)

#Supprimer les deux premières colonnes inutiles
cac40 = cac40.iloc[2:].reset_index(drop=True)

# Afficher les premières lignes du après suppression
cac40.head()

,Price,Adj Close,Close,High,Low,Open
0,1999-12-01,6190.9599609375,6190.9599609375,6396.97021484375,5388.85009765625,6024.3798828125
1,2000-03-01,6426.259765625,6426.259765625,6591.43994140625,5752.43994140625,6272.1298828125
2,2000-06-01,6625.419921875,6625.419921875,6780.66015625,6354.93017578125,6428.4599609375
3,2000-09-01,5928.080078125,5928.080078125,6944.77001953125,5828.14013671875,6648.64013671875
4,2000-12-01,5367.47998046875,5367.47998046875,6126.41015625,5302.7900390625,5942.35986328125


Regression linéaire : prédiction du cac 40 à partir d'indicateurs macroéconomiques

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:





# Étendre les données annuelles en trimestrielles pour l'IPC
ipc["Date"] = pd.to_datetime(ipc["Year"], format="%Y")  # Suppose une colonne 'Year'
ipc = ipc.set_index("Date").resample("Q").ffill().reset_index()

# Fusionner toutes les données dans un seul DataFrame
data = cac40.merge(chomage, on="Date").merge(confiance_trimestriel, on="Date", how="left")
data = data.merge(pib, on="Date").merge(importations, on="Date").merge(exportations, on="Date")
data = data.merge(consommation, on="Date").merge(fbcf, on="Date").merge(ipc, on="Date")

# Supprimer les valeurs manquantes (ou les remplir selon le besoin)
data.dropna(inplace=True)

# Étape 3 : Régression linéaire
X = data.drop(columns=["CAC40", "Date"])  # Variables explicatives
y = data["CAC40"]  # Variable cible

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle de régression
model = LinearRegression()
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Étape 4 : Évaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

# Coefficients du modèle
coefficients = pd.DataFrame({"Feature": X.columns, "Coefficient": model.coef_})
print(coefficients)


KeyError: 'Year'